# Starting kit for PyTorch Deep Learning

Welcome to this tutorial to get started on PyTorch for this competition.
PyTorch is a promising port of Facebook's Torch to Python.

It's only 3 months old but has an already promising feature set.
Unfortunately it's very very raw, and I had a lot of troubles to get started with very basic things:
- data loading
- building a basic CNN
- training

Hopefully this will help you getting started using PyTorch on this dataset.

## Importing libraries
Please note that we do not import numpy but PyTorch wrapper for Numpy

In [1]:
import pandas as pd
from torch import np # Torch wrapper for Numpy

import os
from PIL import Image

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn.preprocessing import MultiLabelBinarizer

## Setting up global variables

In [2]:
IMG_PATH = '../data/train-jpg/'
IMG_EXT = '.jpg'
TRAIN_DATA = '../data/train.csv'

## Loading the data - first part - DataSet

This is probably the most obscure part of PyTorch. Most examples use well known datasets (MNIST ...) and have a custom loader or forces you to have a specific folder structure similar to this:

* data
    * train
          * dogs
          * cats
     * validation
          * dogs
          * cats
     * test
          * test

Data loading in PyTorch is in 2 parts

First the data must be wrapped in a __Dataset__ class with a getitem method that from an index return X_train[index] and y_train[index] and a length method. A Dataset is basically a data storage.

The following solution loads the image name from a CSV and file path + extension and can be adapted easily for most Kaggle challenges. You won't have to write your own ;).

The code will:

- Check that all images in CSV exist in the folder
- Use ScikitLearn MultiLabelBinarizer to OneHotEncode the labels, mlb.inverse_transform(predictions) can be used to get back the textual labels from the predictions
- Apply PIL transformations to the images. See [here](http://pytorch.org/docs/torchvision/transforms.html) for the supported list.
- Use ToTensor() to convert from an image with color scale 0-255 to a Tensor with color scale 0-1.

Note: We use PIL instead of OpenCV because it's Torch default image loader and is compatible with `ToTensor()` method. An fast loader called accimage is currently in development and was published 3 days ago [here](https://github.com/pytorch/accimage).

Note 2: This only provides a mapping to the data, **the data is not loaded in memory at this point**. The next part will show you how to load only what is needed for the batch in memory. This is a huge advantage compared to kernels that must load all images at once.

In [3]:
class KaggleAmazonDataset(Dataset):
    """Dataset wrapping images and target labels for Kaggle - Planet Amazon from Space competition.

    Arguments:
        A CSV file path
        Path to image folder
        Extension of images
        PIL transforms
    """

    def __init__(self, csv_path, img_path, img_ext, transform=None):
    
        tmp_df = pd.read_csv(csv_path)
        assert tmp_df['image_name'].apply(lambda x: os.path.isfile(img_path + x + img_ext)).all(), \
"Some images referenced in the CSV file were not found"
        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df['image_name']
        self.y_train = self.mlb.fit_transform(tmp_df['tags'].str.split()).astype(np.float32)

    def __getitem__(self, index):
        img = Image.open(self.img_path + self.X_train[index] + self.img_ext)
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        return len(self.X_train.index)

In [4]:
transformations = transforms.Compose([transforms.Scale(32),transforms.ToTensor()])

dset_train = KaggleAmazonDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)

## Loading the data - second part - DataLoader

As was said, loading the data is in 2 parts, we provided PyTorch with a data storage, and we have to tell it how to load it. This is done with __DataLoader__

The DataLoader defines how you retrieve the images + labels from the dataset. You can tell it to:

* Set the batch size.
* Shuffle and sample the data randomly, hence implementing __train_test_split__ (check SubsetRandomSampler [here](http://pytorch.org/docs/data.html?highlight=sampler))
* Improve performance by loading data via  separate thread `num_worker` and using `pin_memory` for CUDA. Documentation [here](http://pytorch.org/docs/notes/cuda.html?highlight=dataloader).

In [5]:
train_loader = DataLoader(dset_train,
                          batch_size=256,
                          shuffle=True,
                          num_workers=4 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

## Creating your Neural Network

This is tricky, you need  to compute yourself the in_channels and out_channels of your filters hence the 2304 input for the Dense layer. The first input 3 corresponds to the number of channels of your image, the 17 output corresponds to the number of target labels.

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2304, 256)
        self.fc2 = nn.Linear(256, 17)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.size(0), -1) # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.sigmoid(x)

# model = Net() # On CPU
model = Net().cuda() # On GPU

## Defining your training function

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(async=True), target.cuda(async=True) # On GPU
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

## Training your model

In [9]:
for epoch in range(1, 2):
    train(epoch)

Train Epoch: 1 [0/40479 (0%)]	Loss: 0.692961
Train Epoch: 1 [2560/40479 (6%)]	Loss: 0.688596
Train Epoch: 1 [5120/40479 (13%)]	Loss: 0.682759
Train Epoch: 1 [7680/40479 (19%)]	Loss: 0.676263
Train Epoch: 1 [10240/40479 (25%)]	Loss: 0.668731
Train Epoch: 1 [12800/40479 (31%)]	Loss: 0.659607
Train Epoch: 1 [15360/40479 (38%)]	Loss: 0.650362
Train Epoch: 1 [17920/40479 (44%)]	Loss: 0.635071
Train Epoch: 1 [20480/40479 (50%)]	Loss: 0.613103
Train Epoch: 1 [23040/40479 (57%)]	Loss: 0.584744
Train Epoch: 1 [25600/40479 (63%)]	Loss: 0.545705
Train Epoch: 1 [28160/40479 (69%)]	Loss: 0.500222
Train Epoch: 1 [30720/40479 (75%)]	Loss: 0.443742
Train Epoch: 1 [33280/40479 (82%)]	Loss: 0.404282
Train Epoch: 1 [35840/40479 (88%)]	Loss: 0.361815
Train Epoch: 1 [38400/40479 (94%)]	Loss: 0.338667


# Thank you for your attention

Hopefully that will help you get started. I still have a lot to figure out in PyTorch like:

* Implementing the train / validation split
* Figure out data augmentation (and not just random transformations or images)
* Implementing early stopping
* Automating computation of intermediate layers
* Improving the display of each epochs

If you liked the kernel don't forget to vote and don't hesitate to comment.